In [1]:
import sys
import os
from glob import glob
import numpy as np
import pandas as pd
from scipy import ndimage
from scipy.spatial.distance import cdist
from IPython.display import display
from brainsmash.mapgen import Base, Sampled
import nibabel as nib
from tqdm.auto import tqdm
from neuromaps.parcellate import Parcellater
from neuromaps.images import load_nifti
from neuromaps import images
import nilearn as nl
sys.path.append(os.path.dirname('/Users/llotter/projects/cmc/cmc'))
from cmc.api import CMC

/Users/llotter/opt/anaconda3/envs/domanalysis/lib/python3.8/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


# volumetric data

In [2]:
x = glob("/Users/llotter/projects/cmc/testing/test_x/*")
img = x[0]
parcellation = glob("/Users/llotter/projects/cmc/testing/test_parc/*")[0]

## get data
parc = load_nifti(parcellation)
parc_data = parc.get_fdata()
parc_affine = parc.affine
parcels = np.trim_zeros(np.unique(parc_data))
n_parcels = len(parcels)
parcellater = Parcellater(parc, "MNI152", resampling_target="data")
img = load_nifti(img)
img_parc = parcellater.fit_transform(img, "MNI152")

In [12]:
## distance matrix, from neuromaps

mask = np.logical_not(np.logical_or(np.isclose(parc_data, 0), np.isnan(parc_data)))
ijk = nib.affines.apply_affine(parc_affine, np.column_stack(np.where(mask)))
parc_data_m = parc_data[mask]

row_dist = np.zeros((len(ijk), n_parcels), dtype='float32')
dist = np.zeros((n_parcels, n_parcels), dtype='float32')
for n, row in enumerate(tqdm(ijk, desc="xyz loop")):
    ijk_dist = cdist(row[None], ijk).astype('float32')
    row_dist[n] = ndimage.mean(ijk_dist, parc_data_m, parcels)
for n in range(n_parcels):
    dist[n] = ndimage.mean(row_dist[:, n], parc_data_m, parcels)
display(pd.DataFrame(dist))

xyz loop:   0%|          | 0/139942 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,15.151454,46.026604,33.721268,65.145287,65.707100,43.989674,53.251812,68.881470,68.916901,42.830582,...,66.535881,66.289368,19.624897,31.298292,27.342598,39.001465,52.012741,35.743858,40.444614,55.000160
1,46.026604,16.820744,25.400406,26.864336,30.750854,31.447741,35.893322,40.199345,46.549068,66.543327,...,95.429375,99.582245,56.178623,74.047806,55.014908,69.409180,93.130302,74.907661,79.371223,92.382347
2,33.721268,25.400406,16.829594,39.798676,35.495369,22.355152,39.674416,43.678761,42.380585,57.434650,...,77.080132,80.795021,42.278198,59.623802,37.336876,51.280720,76.184692,58.256878,63.544510,74.812157
3,65.145287,26.864336,39.798676,19.611647,27.394293,39.231438,37.701271,32.039948,42.510342,79.706429,...,110.774406,115.125069,74.796707,93.203796,71.124176,84.850189,111.045380,92.392075,96.459793,108.571716
4,65.707100,30.750854,35.495369,27.394293,14.368005,31.985382,50.473072,35.107979,33.330921,86.550995,...,98.874741,106.031105,74.691216,92.009216,66.761063,79.218971,107.026978,89.799530,95.423447,104.458740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,39.001465,69.409180,51.280720,84.850189,79.218971,52.453995,69.468933,78.167923,70.132301,48.834442,...,41.230312,35.409554,33.553623,34.256222,18.331335,13.358422,33.381077,21.054539,27.783798,27.983427
112,52.012741,93.130302,76.184692,111.045380,107.026978,81.020866,94.426453,107.634491,101.332047,59.429459,...,42.061672,30.843819,42.045494,26.613182,42.742783,33.381077,8.684171,21.064293,23.247938,18.319929
113,35.743858,74.907661,58.256878,92.392075,89.799530,63.043461,74.635872,88.701996,83.864815,43.461880,...,46.984619,38.468670,26.968555,19.509529,25.321857,21.054539,21.064293,8.764838,15.420214,22.352907
114,40.444614,79.371223,63.544510,96.459793,95.423447,68.219154,76.098236,92.005592,88.366791,40.704655,...,54.027538,43.424641,31.500271,23.867081,31.801302,27.783798,23.247938,15.420214,15.517078,23.385948


In [15]:
dist[0,:10]

array([15.151454, 46.026604, 33.721268, 65.14529 , 65.7071  , 43.989674,
       53.251812, 68.88147 , 68.9169  , 42.83058 ], dtype=float32)

In [22]:
## distance matrix, speed improved, one triangle

mask = np.logical_not(np.logical_or(np.isclose(parc_data, 0), np.isnan(parc_data)))
parc_data_m = parc_data * mask

ijk_parcels = dict()
for i_parcel in parcels:
    xyz_parcel = np.column_stack(np.where(parc_data_m==i_parcel))
    ijk_parcels[i_parcel] = nib.affines.apply_affine(parc_affine, xyz_parcel)

dist = np.zeros((n_parcels, n_parcels), dtype='float32')
for i, i_parcel in enumerate(tqdm(parcels)):
    j = i
    for _ in range(n_parcels - j):
        dist[i,j] = cdist(ijk_parcels[i_parcel], ijk_parcels[parcels[j]]).mean().astype('float32')
        j += 1

dist = dist + dist.T - np.diag(np.diag(dist))

dist[0,:10]

  0%|          | 0/116 [00:00<?, ?it/s]

array([15.151454, 46.026604, 33.721268, 65.14529 , 65.7071  , 43.989674,
       53.251812, 68.88147 , 68.9169  , 42.83058 ], dtype=float32)

In [17]:
## distance matrix, parcel centroids

mask = np.logical_not(np.logical_or(np.isclose(parc_data, 0), np.isnan(parc_data)))
parc_data_m = parc_data * mask

xyz = np.zeros((n_parcels, 3), float)
for i, i_parcel in enumerate(parcels):
    xyz[i,:] = np.column_stack(np.where(parc_data_m==i_parcel)).mean(axis=0)
ijk = nib.affines.apply_affine(parc_affine, xyz)

dist = np.zeros((n_parcels, n_parcels), dtype='float32')
for i, row in enumerate(ijk):
    dist[i] = cdist(row[None], ijk).astype('float32')
    
dist[0,:10]

array([ 0.      , 44.091846, 31.6248  , 63.11293 , 64.568054, 42.52424 ,
       51.843227, 67.06589 , 67.84859 , 38.684334], dtype=float32)

In [21]:
gen = Base(x=img_parc[0,], D=dist, seed=42, resample=False)
nulls = gen(1000, 100)

In [14]:
nulls.shape

(1000, 116)

## test

In [37]:
from cmc.nulls import vol_distance_matrix
vol_distance_matrix(parcellation)

  0%|          | 0/116 [00:00<?, ?it/s]

# surface data

In [3]:
destrieux = nl.datasets.fetch_atlas_surf_destrieux()
destrieux_labels = [l.decode() for l in destrieux['labels']]
# make labels to fit braincharts model IDPs
destrieux_idps = [l for l in destrieux_labels if l not in ['Unknown', 'Medial_wall']]
destrieux_idps = ['lh_' + l.replace('_and_', '&') + '_thickness' for l in destrieux_idps] + \
                 ['rh_' + l.replace('_and_', '&') + '_thickness' for l in destrieux_idps]
# make gifti from parcellations vectors
parc_left = images.construct_shape_gii(destrieux['map_left'], labels=destrieux_labels, intent='NIFTI_INTENT_LABEL')
parc_right = images.construct_shape_gii(destrieux['map_right'], labels=destrieux_labels, intent='NIFTI_INTENT_LABEL')
parcellation = images.relabel_gifti((parc_left, parc_right), background=['Medial_wall'])

from neuromaps.nulls import burt2020
data_surf = np.random.rand(len(destrieux_idps))

In [28]:
## distance matrix neuromaps
from neuromaps.nulls.nulls import _get_distmat

parc_hemi = None

if parc_hemi is None:
    parc_hemi = ["L", "R"]

dist = list()
for i_hemi, hemi in enumerate(parc_hemi):
    dist.append(_get_distmat(hemi, 
                             atlas="fsaverage", 
                             density="10k", 
                             parcellation=parcellation[i_hemi],
                             n_proc=-1))
dist = tuple(dist)

In [33]:
## burt 2020 with precomputed dist mat
nulls = burt2020(data=data_surf,
                 parcellation=parcellation,
                 atlas="fsaverage",
                 density="10k",
                 distmat=dist,
                 n_perm=1000,
                 seed=42
                 ).T

# Test

In [3]:
from cmc.nulls import generate_null_maps

data_vol = np.random.random([10,116])
data_surf = np.random.random([10,148])

# vol
nullsvol = generate_null_maps(data=data_vol,
                              parcellation=parc)

# surf
nullsurf = generate_null_maps(data=data_surf,
                              parcellation=parcellation)

INFO:cmc.nulls:Null map generation: Assuming n = 10 data vector(s) for n = 116 parcels.
INFO:cmc.nulls:Calculating distance matrix/matrices (space = 'MNI152')


Generating null data:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:cmc.nulls:Null data generation finished.
INFO:cmc.nulls:Null map generation: Assuming n = 10 data vector(s) for n = 148 parcels.
INFO:cmc.nulls:Calculating distance matrix/matrices (space = 'MNI152')


Generating null data:   0%|          | 0/10 [00:00<?, ?it/s]

ValueError: Distance matrix must have dimensions consistent with brain map
Distance matrix shape: (116, 116)
Brain map size: 148

In [4]:
nullsvol

({0: array([[-0.10420287,  0.52256725,  0.12336125, ...,  0.28569609,
           0.52256313,  0.13689629],
         [ 0.08504543, -0.1140205 ,  0.2078417 , ...,  0.13067392,
           0.14410898,  0.30016059],
         [ 0.08612203, -0.07581711, -0.0330339 , ..., -0.01355356,
           0.11150103,  0.13913664],
         ...,
         [ 0.53850028,  0.38324876,  0.36951426, ...,  0.25133923,
          -0.109461  , -0.57627549],
         [ 0.28046373,  0.27413833,  0.32466452, ...,  0.19689478,
          -0.10733851, -0.06226131],
         [ 0.37726571, -0.28758603, -0.08939461, ...,  0.46585325,
           0.16101567, -0.27597946]]),
  1: array([[-0.54368134, -0.67972526,  0.22688211, ...,  0.40036343,
           0.50207068, -0.40463923],
         [ 0.53287908,  1.20150056,  0.09336765, ..., -0.31436545,
          -0.60457305, -0.26334218],
         [-0.07979624,  0.53744257,  0.46256689, ..., -0.63447648,
          -0.26546394, -1.09428444],
         ...,
         [ 0.48862787,  0.97